# 3단계, 데이터 학습 전 준비과정(Prepate data for Consumption)
***


## 눈여겨 볼것

- 파이썬의 할당, 같다 라는 개념은 참조인지, 값인지가 다르기 때문에 copy 함수를 사용하기로 한다.
- copy function: https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
- data_raw.describe(include = 'all')
- dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0
- dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)
- data1['Title'].value_counts()
- title_names = (data1['Title'].value_counts() < stat_min)
- data_val.info()
- data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
- lambda function

***

## 3.1 라이브러리 및 패키지 불러오기 (Import Libraries)

In [2]:
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

#패키지 불러오기
import sys #시스템 환경에 접근하도록 돕는 패키지 (https://docs.python.org/3/library/sys.html)
print("Python version: {}". format(sys.version)) # 버전 출력

import pandas as pd #데이터프레임을 다루기 쉽게 도와주는 패키지
print("pandas version: {}". format(pd.__version__)) # 버전 출력

import matplotlib #데이터 시각화를 돕기 위한 패키지
print("matplotlib version: {}". format(matplotlib.__version__)) # 버전 출력

import numpy as np #과학 계산의 기반이 되는 패키지
print("NumPy version: {}". format(np.__version__)) # 버전 출력

import scipy as sp #과학계산과 심화수학 계산을 돕는 패키지
print("SciPy version: {}". format(sp.__version__))  # 버전 출력

import IPython
from IPython import display #Jupyter notebook 활용을 위한 패키지
print("IPython version: {}". format(IPython.__version__))  # 버전 출력

import sklearn #머신러닝 알고리즘 묶음 패키지
print("scikit-learn version: {}". format(sklearn.__version__)) # 버전 출력

#기타 내장 라이브러리
import random
import time


#경고문 무시하기
import warnings
warnings.filterwarnings('ignore')
print('-'*25)



# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# from subprocess import check_output
# print(check_output(["ls", "./data/"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

Python version: 3.6.5 |Anaconda, Inc.| (default, Mar 29 2018, 13:32:41) [MSC v.1900 64 bit (AMD64)]
pandas version: 0.22.0
matplotlib version: 2.1.2
NumPy version: 1.14.2
SciPy version: 1.0.1
IPython version: 6.3.1
scikit-learn version: 0.19.1
-------------------------


## 3.11 학습시킬 모델 불러오기 (Load Data Modeling Libraries)

In [3]:
#일반적인 학습 알고리즘 불러오기
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, ensemble, discriminant_analysis, gaussian_process
# from xgboost import XGBClassifier

#일반적인 학습모델 도움을 주는 도구
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics

#시각화 도구
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

#시각화 기본설정하기
#%matplotlib inline = Jupyter Notebook 환경에서 보여주게끔 설정
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

## 3.2 Meet and Greet Data

1. The Survived variable is our outcome or dependent variable. It is a binary nominal datatype of 1 for survived and 0 for did not survive. All other variables are potential predictor or independent variables. It's important to note, more predictor variables do not make a better model, but the right variables.
2. The PassengerID and Ticket variables are assumed to be random unique identifiers, that have no impact on the outcome variable. Thus, they will be excluded from analysis.
3. The Pclass variable is an ordinal datatype for the ticket class, a proxy for socio-economic status (SES), representing 1 = upper class, 2 = middle class, and 3 = lower class.
4. The Name variable is a nominal datatype. It could be used in feature engineering to derive the gender from title, family size from surname, and SES from titles like doctor or master. Since these variables already exist, we'll make use of it to see if title, like master, makes a difference.
5. The Sex and Embarked variables are a nominal datatype. They will be converted to dummy variables for mathematical calculations.
6. The Age and Fare variable are continuous quantitative datatypes.
7. The SibSp represents number of related siblings/spouse aboard and Parch represents number of related parents/children aboard. Both are discrete quantitative datatypes. This can be used for feature engineering to create a family size and is alone variable.
8. The Cabin variable is a nominal datatype that can be used in feature engineering for approximate position on ship when the incident occurred and SES from deck levels. However, since there are many null values, it does not add value and thus is excluded from analysis.

In [4]:
#데이터 불러오기 (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
data_raw = pd.read_csv('./data/train.csv')


#데이터셋은 다음과 같이 나눠져야 한다.: train, test, and (final) validation
#이번 컴피티션에선 최종 업로드를 하면 사이트에서 validation 즉 유효성 검증 단계가 진행된다.
#우리는 추후에 train 데이터를 train, test로 다시금 나누려고 한다. 
data_val  = pd.read_csv('./data/test.csv')


#우선 복사를 해두는 것이 좋다.
#파이썬의 할당, 같다 라는 개념은 참조인지, 값인지가 다르기 때문에 copy 함수를 사용하기로 한다.
#copy function: https://stackoverflow.com/questions/46327494/python-pandas-dataframe-copydeep-false-vs-copydeep-true-vs
data1 = data_raw.copy(deep = True)

#하지만 참조해서 후에 데이터 전처리를 잘하기 위해서 묶어주는 것이 좋다.
data_cleaner = [data1, data_val]


#데이터 먼저 보기
print (data_raw.info()) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB
None


In [5]:
data_raw.head() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.head.html

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
data_raw.tail() #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.tail.html

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [7]:
data_raw.sample(10) #https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.sample.html

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
482,483,0,3,"Rouse, Mr. Richard Henry",male,50.0,0,0,A/5 3594,8.0500,NaN,S
144,145,0,2,"Andrew, Mr. Edgardo Samuel",male,18.0,0,0,231945,11.5000,NaN,S
391,392,1,3,"Jansson, Mr. Carl Olof",male,21.0,0,0,350034,7.7958,NaN,S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S
304,305,0,3,"Williams, Mr. Howard Hugh ""Harry""",male,NaN,0,0,A/5 2466,8.0500,NaN,S
59,60,0,3,"Goodwin, Master. William Frederick",male,11.0,5,2,CA 2144,46.9000,NaN,S
731,732,0,3,"Hassan, Mr. Houssein G N",male,11.0,0,0,2699,18.7875,NaN,C
480,481,0,3,"Goodwin, Master. Harold Victor",male,9.0,5,2,CA 2144,46.9000,NaN,S
587,588,1,1,"Frolicher-Stehli, Mr. Maxmillian",male,60.0,1,1,13567,79.2000,B41,C
338,339,1,3,"Dahl, Mr. Karl Edwart",male,45.0,0,0,7598,8.0500,NaN,S


## 3.21 The 4C's of Data Cleaning: Correcting, Completing, Creating, and Converting
해당 단계에서 데이터를 깨끗하게 클렌징해주고자 한다. 

1단계. Correcting<br>
 :이상값, Outlier 값을 바로잡아준다. 나이가 900세이거나 승선료가 다른 인원보다 월등히 높은 경우가 예시다. 나이가 900세인 경우는 존재할 수 없기 때문에 지워주는게 맞지만 평균에서 지나치게 떨어져 있는 Outlier의 경우, 때로는 정확한 모델을 위해 필요할 수 있기 때문에 실험해보는 게 좋다.<br>

 
2단계. Completing<br>
 : 결측치에 대한 대응을 한다. 비어있는 null 값이나, 아예 없는 값들에 대한 대응이다. null 값을 모델이 대응할 수 없기 때문에 null 값을 어떻게 변환하거나 소거해줄지는 매우 중요하다. 일단 결측치를 소거해주는 건 좋은 대응은 아니고 최대한 유추해서 넣어주는 작업이 좋다. mode 즉 최빈값을 넣어주는 게 기본적으론 좋은 방법이다. 중앙값(중위수), 평균, 평균 + 임의의 표준편차를 넣는 것도 마찬가지다. 다른 복잡한 방법도 많지만 기본적인 방법을 놓고 조금씩 변용하는 게 옳다.<br>
 
 
3단계. Creating<br>
 : 분석을 위한 새로운 Feature를 뽑아낸다. 이를 보통 Feature Engeering이라고 하는데 결과값에 영향을 주는 유의미한 Feature 특징을 뽑아낼 때다.<br>
 
 
4단계. Converting<br>
 : 계산을 위해 타당한 형으로 변환해준다.마지막 단계다. 컴퓨터가 이해할 수 있게끔 object datatype을 1,0으로 이뤄진 categorical dummy variables로 변환해준다.<br>

In [9]:
print('Train columns with null values:\n', data1.isnull().sum())
print("-"*10)

print('Test/Validation columns with null values:\n', data_val.isnull().sum())
print("-"*10)

data_raw.describe(include = 'all')

Train columns with null values:
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
----------
Test/Validation columns with null values:
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64
----------


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
count,891.000000,891.000000,891.000000,891,891,714.000000,891.000000,891.000000,891,891.000000,204,889
unique,NaN,NaN,NaN,891,2,NaN,NaN,NaN,681,NaN,147,3
top,NaN,NaN,NaN,"Parkes, Mr. Francis ""Frank""",male,NaN,NaN,NaN,347082,NaN,C23 C25 C27,S
freq,NaN,NaN,NaN,1,577,NaN,NaN,NaN,7,NaN,4,644
mean,446.000000,0.383838,2.308642,NaN,NaN,29.699118,0.523008,0.381594,NaN,32.204208,NaN,NaN
std,257.353842,0.486592,0.836071,NaN,NaN,14.526497,1.102743,0.806057,NaN,49.693429,NaN,NaN
min,1.000000,0.000000,1.000000,NaN,NaN,0.420000,0.000000,0.000000,NaN,0.000000,NaN,NaN
25%,223.500000,0.000000,2.000000,NaN,NaN,20.125000,0.000000,0.000000,NaN,7.910400,NaN,NaN
50%,446.000000,0.000000,3.000000,NaN,NaN,28.000000,0.000000,0.000000,NaN,14.454200,NaN,NaN
75%,668.500000,1.000000,3.000000,NaN,NaN,38.000000,1.000000,0.000000,NaN,31.000000,NaN,NaN


## 3.22 Clean Data

In [13]:
# 채워넣거나 아예 결측치를 지워버리는 과정

for dataset in data_cleaner:
    # 나이 결측치를 중위값으로 채워넣는다.
    dataset['Age'].fillna(dataset['Age'].median(), inplace = True)
    # 정박항인 'Embarked'는 최빈값으로 대체한다.
    dataset['Embarked'].fillna(dataset['Embarked'].mode(), inplace = True)
    # 운항료 정보는 중앙값으로 대체한다.
    dataset['Fare'].fillna(dataset['Fare'].median(), inplace = True)
    
# 객실 번호, 승객번호, 티켓번호 는 무의미한 데이터기 때문에 소거한다.

drop_columns =  ['PassengerId','Cabin','Ticket']
data1.drop(drop_columns, axis =1, inplace = True)

print(data1.isnull().sum())
print("-"*10)
print(data_val.isnull().sum())

Survived    0
Pclass      0
Name        0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    2
dtype: int64
----------
PassengerId      0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


In [14]:
# Create: 새로운 데이터 특징을 뽑아낸다.

for dataset in data_cleaner:
    # 이산변수들
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
    
    dataset['IsAlone'] = 1 # 기본적으로는 맞다 라고 선언
    dataset['IsAlone'].loc[dataset['FamilySize'] > 1] = 0 # 가족 사이즈가 1보다 큰 인원들은 아니라고 선언해준다.
    
    #Split을 활용하여 이름에서 칭호(title)을 뽑아준다. 
    # expand 는 column을 확장할 것인가 묻는 것
    dataset['Title'] = dataset['Name'].str.split(", ", expand=True)[1].str.split(".", expand=True)[0]
    
    #연속형 변수; qcut vs cut: https://stackoverflow.com/questions/30211923/what-is-the-difference-between-pandas-qcut-and-pandas-cut
    #빈도에 따라서 사분위수를 나눠서 명시한다. bins: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.qcut.html
    dataset['FareBin'] = pd.qcut(dataset['Fare'], 4)

    #나이는 cut으로 대응한다.: https://pandas.pydata.org/pandas-docs/stable/generated/pandas.cut.html
    # cut은 최대값과 최소값을 기준으로 사분위수를 나누는 것
    dataset['AgeBin'] = pd.cut(dataset['Age'].astype(int), 5)


In [17]:
# 다양한 호칭들이 나오는 것을 볼 수 있다.
all_of_title = data1['Title'].value_counts()
print(all_of_title)

Mr              517
Miss            182
Mrs             125
Master           40
Dr                7
Rev               6
Major             2
Col               2
Mlle              2
Capt              1
Sir               1
Ms                1
Mme               1
Don               1
Lady              1
Jonkheer          1
the Countess      1
Name: Title, dtype: int64


In [18]:
stat_min = 10 #작다는 기준은 통계학에서 주로 쓰는 10으로 설정: http://nicholasjjackson.com/2012/03/08/sample-size-is-10-a-magic-number/
title_names = (data1['Title'].value_counts() < stat_min) #기준으로 True, False가 설정된다. 

#apply and lambda functions are quick and dirty code to find and replace with fewer lines of code: https://community.modeanalytics.com/python/tutorial/pandas-groupby-and-python-lambda-functions/
data1['Title'] = data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)
print(data1['Title'].value_counts())
print("-"*10)


#데이터를 다시한번 확인한다.
data1.info()
data_val.info()
data1.sample(10)

Mr        517
Miss      182
Mrs       125
Master     40
Misc       27
Name: Title, dtype: int64
----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Name          891 non-null object
Sex           891 non-null object
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Embarked      889 non-null object
FamilySize    891 non-null int64
IsAlone       891 non-null int64
Title         891 non-null object
FareBin       891 non-null category
AgeBin        891 non-null category
dtypes: category(2), float64(2), int64(6), object(4)
memory usage: 85.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 16 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null 

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,FamilySize,IsAlone,Title,FareBin,AgeBin
366,1,1,"Warren, Mrs. Frank Manley (Anna Sophia Atkinson)",female,60.0,1,0,75.25,C,2,0,Mrs,"(31.0, 512.329]","(48.0, 64.0]"
565,0,3,"Davies, Mr. Alfred J",male,24.0,2,0,24.15,S,3,0,Mr,"(14.454, 31.0]","(16.0, 32.0]"
236,0,2,"Hold, Mr. Stephen",male,44.0,1,0,26.00,S,2,0,Mr,"(14.454, 31.0]","(32.0, 48.0]"
538,0,3,"Risien, Mr. Samuel Beard",male,28.0,0,0,14.50,S,1,1,Mr,"(14.454, 31.0]","(16.0, 32.0]"
245,0,1,"Minahan, Dr. William Edward",male,44.0,2,0,90.00,Q,3,0,Misc,"(31.0, 512.329]","(32.0, 48.0]"
351,0,1,"Williams-Lambert, Mr. Fletcher Fellows",male,28.0,0,0,35.00,S,1,1,Mr,"(31.0, 512.329]","(16.0, 32.0]"
133,1,2,"Weisz, Mrs. Leopold (Mathilde Francoise Pede)",female,29.0,1,0,26.00,S,2,0,Mrs,"(14.454, 31.0]","(16.0, 32.0]"
282,0,3,"de Pelsmaeker, Mr. Alfons",male,16.0,0,0,9.50,S,1,1,Mr,"(7.91, 14.454]","(-0.08, 16.0]"
330,1,3,"McCoy, Miss. Agnes",female,28.0,2,0,23.25,Q,3,0,Miss,"(14.454, 31.0]","(16.0, 32.0]"
308,0,2,"Abelson, Mr. Samuel",male,30.0,1,0,24.00,C,2,0,Mr,"(14.454, 31.0]","(16.0, 32.0]"
